In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from konlpy.tag import Okt
from collections import Counter
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from keras.callbacks import ModelCheckpoint
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer

# 모델 저장위한 라이브러리
from keras.utils import np_utils
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation
from numpy import argmax
from keras.models import load_model


import pandas as pd
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

In [4]:
import warnings
warnings.filterwarnings('ignore')

# 데이터 수집
df = pd.read_csv("./data/책종합.csv")

# 모델을 저장할 폴더
# MODEL_DIR = './model/'

# 전처리
df = df.dropna(how='any')
# 불필요한 컬럼 삭제
df.drop('book_seq', axis=1, inplace=True)
df.drop('book_isbn', axis=1, inplace=True)
df.drop('author', axis=1, inplace=True)
df.drop('publisher', axis=1, inplace=True)
df.drop('book_salecount', axis=1, inplace=True)
df.drop('book_price', axis=1, inplace=True)
df.drop('book_genre', axis=1, inplace=True)
df.drop('book_link', axis=1, inplace=True)
df.drop('book_bestdate', axis=1, inplace=True)
df.drop('book_image1', axis=1, inplace=True)
df.drop('book_image2', axis=1, inplace=True)

# 카테고리별 데이터프레임 생성
categorical = df['book_category'].unique()

cate_ = []
for i in range(len(categorical)):
    cate = []
    cate = df['book_category'] == categorical[i]
    cate_.append( df[cate])

# 카테고리별 데이터프레임 인덱스 재정렬
for i in range(len(cate_)):
    cate_[i] = cate_[i].reset_index(drop=True)
    
# k = 0:건강 취미 1:예술 2:인문 3:자기계발 4:에세이 5:종교 6:소설/시/희곡
for k in range(len(cate_)):
    
#     modelpath = MODEL_DIR + 'category' + k
    
    # 공백과 한글 제외 제거하는 함수
    def apply_regular_expression(book_intro):
        hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 한글 추출 규칙: 띄어 쓰기(1 개)를 포함한 한글
        result = hangul.sub('', book_intro)  # 위에 설정한 "hangul"규칙을 "text"에 적용(.sub)시킴
        return result

    okt = Okt()  # 명사 형태소 추출 함수
    # nouns = okt.nouns(apply_regular_expression(cate_[0]['book_intro'][0]))
    nouns = []

    # 명사분류
    for i in range(10):
        nouns.append(okt.nouns(apply_regular_expression(cate_[k]['book_intro'][i])))

    # 말뭉치 생성
    corpus = "".join(cate_[k]['book_intro'].tolist())

    nouns = okt.nouns(apply_regular_expression(corpus))

    # 단어 횟수만큼 카운트
    counter = Counter(nouns)

    # 한글자 삭제
    available_counter = Counter({x: counter[x] for x in counter if len(x) > 1})
    available_counter.most_common(10)

    # 불용어 삭제
    stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt").values.tolist()
    stopwords[:10]

    # 불용어 외로 더 지워도 되는건 지우기
    jeju_hotel_stopwords = ['저자', '작가', '사람', '제주', '토박이', '블로거', '코로나']
    for word in jeju_hotel_stopwords:
        stopwords.append(word)


    def text_cleaning(book_intro):
        hangul = re.compile('[^ ㄱ-ㅣ 가-힣]')  # 정규 표현식 처리
        result = hangul.sub('', book_intro)
        okt = Okt()  # 형태소 추출
        nouns = okt.nouns(result)
        nouns = [x for x in nouns if len(x) > 1]  # 한글자 키워드 제거
        nouns = [x for x in nouns if x not in stopwords]  # 불용어 제거
        return nouns

    vect = CountVectorizer(tokenizer = lambda x: text_cleaning(x))
    bow_vect = vect.fit_transform(cate_[k]['book_intro'].tolist())
    word_list = vect.get_feature_names()
    count_list = bow_vect.toarray().sum(axis=0)

    word_count_dict = dict(zip(word_list, count_list))

    tfidf_vectorizer = TfidfTransformer()
    tf_idf_vect = tfidf_vectorizer.fit_transform(bow_vect)

    invert_index_vectorizer = {v: k for k, v in vect.vocabulary_.items()}
    
    # 책 평점이 9.5 이상은 긍정, 아니면 부정으로 분류
    def rating_to_label(book_score):
        if book_score > 9.5:
            return 1
        else:
            return 0

    # 긍/부정 나눈 라벨 값 삽입
    cate_[k]['y'] = cate_[k]['book_score'].apply(lambda x: rating_to_label(x))

    # 해당 카테고리별 데이터프레임 따로 저장
    cate_[k].to_csv('{}bookmodel.csv'.format(k+1), index=False, encoding="utf-8")

    # 해당 데이터프레임에서 트레인과 훈련셋 나누기
    x = tf_idf_vect
    # y = df['y']
    y = cate_[k]['y']
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=1)
    
    # 모델학습 들어가기
    model = models.Sequential()
    model.add( Dense(5000, input_dim=x_train.shape[1], activation='relu') ) # x_train.shape, x_test.shape로 확인해서 input_dim 설정
    model.add(layers.Dense(4500, activation='relu'))
    model.add(layers.Dense(4000, activation='relu'))
    model.add(layers.Dense(3500, activation='relu'))
    model.add(layers.Dense(3000, activation='relu'))
    model.add(layers.Dense(2500, activation='relu'))
    model.add(layers.Dense(2000, activation='relu'))
    model.add(layers.Dense(1500, activation='relu'))
    model.add(layers.Dense(1000, activation='relu'))
    model.add(layers.Dense(500, activation='relu'))
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    #모델 생성
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    # fit in training set
    lr = LogisticRegression(random_state = 0)
    lr.fit(x_train, y_train)

    # predict in test set
    y_pred = lr.predict(x_test)

    confu = confusion_matrix(y_true = y_test, y_pred = y_pred)

#     positive_random_idx = cate_[k][cate_[k]['y']==1].sample(75, random_state=12).index.tolist()
#     negative_random_idx = cate_[k][cate_[k]['y']==0].sample(75, random_state=12).index.tolist()

#     random_idx = positive_random_idx + negative_random_idx
#     x = tf_idf_vect[random_idx]
#     y = cate_[k]['y'][random_idx]    
    
#     # 정수 인코딩
#     max_feature = 35000
#     # 상위 35,000단어 보존
#     tokenizer = Tokenizer(num_words=max_feature)
#     # 단어에 인덱스 부여하기 위해
#     tokenizer.fit_on_texts(x_train)
    
    x_train_a = x_train.toarray()
    x_test_a = x_test.toarray()
    # 모델 학습
    model.fit(x_train_a, y_train, epochs=1000, batch_size=100, validation_data=(x_test_a,y_test))
    
#     loss_and_metrics = model.evaluate(x_test, y_test, batch_size=32)
    
    # 모델 저장
    model.save('{}bookmodel.h5'.format(k+1))  

Epoch 1/1000
6/6 [==============================] - 5s 678ms/step - loss: 0.6952 - accuracy: 0.4828 - val_loss: 0.6934 - val_accuracy: 0.4866
Epoch 2/1000
6/6 [==============================] - 4s 633ms/step - loss: 0.6929 - accuracy: 0.5211 - val_loss: 0.6941 - val_accuracy: 0.4866
Epoch 3/1000
6/6 [==============================] - 4s 641ms/step - loss: 0.6924 - accuracy: 0.5211 - val_loss: 0.6926 - val_accuracy: 0.4866
Epoch 4/1000
6/6 [==============================] - 4s 682ms/step - loss: 0.5764 - accuracy: 0.5211 - val_loss: 0.7499 - val_accuracy: 0.4866
Epoch 5/1000
6/6 [==============================] - 4s 657ms/step - loss: 0.3724 - accuracy: 0.8678 - val_loss: 1.3190 - val_accuracy: 0.5357
Epoch 6/1000
6/6 [==============================] - 4s 654ms/step - loss: 0.2046 - accuracy: 0.9540 - val_loss: 0.7638 - val_accuracy: 0.5357
Epoch 7/1000
6/6 [==============================] - 4s 648ms/step - loss: 0.0732 - accuracy: 0.9866 - val_loss: 3.6645 - val_accuracy: 0.5714
Epoch 